In [70]:
import pandas as pd
import yfinance as yf
import pandas_ta as ts

In [71]:
# Sample Inputs
investment_amount = 10000  # Total investment amount
start_date = "2024-01-01"  # Investment period start date
end_date = "2024-05-01"  # Investment period end date

In [72]:
stocks = [
    {"Ticker": "ADANIPORTS.NS", "Weightage": 0.0082},
    # {"Ticker": "APOLLOHOSP.NS", "Weightage": 0.0061},
    # {"Ticker": "ASIANPAINT.NS", "Weightage": 0.0195},
    # {"Ticker": "AXISBANK.NS", "Weightage": 0.0257},
    # {"Ticker": "BAJAJFINSV.NS", "Weightage": 0.012},
    # {"Ticker": "BAJFINANCE.NS", "Weightage": 0.0237},
    # {"Ticker": "BHARTIARTL.NS", "Weightage": 0.0233},
    # {"Ticker": "BPCL.NS", "Weightage": 0.0046},
    # {"Ticker": "BRITANNIA.NS", "Weightage": 0.0052},
    # {"Ticker": "CIPLA.NS", "Weightage": 0.0068},
    # {"Ticker": "COALINDIA.NS", "Weightage": 0.0051},
    # {"Ticker": "DIVISLAB.NS", "Weightage": 0.0077},
    # {"Ticker": "DRREDDY.NS", "Weightage": 0.0067},
    # {"Ticker": "EICHERMOT.NS", "Weightage": 0.0049},
    # {"Ticker": "GRASIM.NS", "Weightage": 0.0085},
    # {"Ticker": "HCLTECH.NS", "Weightage": 0.0153},
    # {"Ticker": "HDFCBANK.NS", "Weightage": 0.081},
    # {"Ticker": "HDFCLIFE.NS", "Weightage": 0.0072},
    # {"Ticker": "HEROMOTOCO.NS", "Weightage": 0.0043},
    # {"Ticker": "HINDALCO.NS", "Weightage": 0.0094},
    # {"Ticker": "HINDUNILVR.NS", "Weightage": 0.0267},
    # {"Ticker": "ICICIBANK.NS", "Weightage": 0.069},
    # {"Ticker": "INDUSINDBK.NS", "Weightage": 0.0085},
    # {"Ticker": "INFY.NS", "Weightage": 0.0766},
    # {"Ticker": "ITC.NS", "Weightage": 0.0303},
    # {"Ticker": "JSWSTEEL.NS", "Weightage": 0.0094},
    # {"Ticker": "KOTAKBANK.NS", "Weightage": 0.0351},
    # {"Ticker": "LT.NS", "Weightage": 0.0274},
    # {"Ticker": "MARUTI.NS", "Weightage": 0.0137},
    # {"Ticker": "NESTLEIND.NS", "Weightage": 0.0087},
    # {"Ticker": "NTPC.NS", "Weightage": 0.0099},
    # {"Ticker": "ONGC.NS", "Weightage": 0.0078},
    # {"Ticker": "POWERGRID.NS", "Weightage": 0.0104},
    # {"Ticker": "RELIANCE.NS", "Weightage": 0.1286},
    # {"Ticker": "SBILIFE.NS", "Weightage": 0.0065},
    # {"Ticker": "SBIN.NS", "Weightage": 0.0254},
    # {"Ticker": "SHREECEM.NS", "Weightage": 0.0046},
    # {"Ticker": "SUNPHARMA.NS", "Weightage": 0.0134},
    # {"Ticker": "TATACONSUM.NS", "Weightage": 0.0066},
    # {"Ticker": "TATAMOTORS.NS", "Weightage": 0.0105},
    # {"Ticker": "TATASTEEL.NS", "Weightage": 0.0137},
    # {"Ticker": "TCS.NS", "Weightage": 0.0491},
    # {"Ticker": "TECHM.NS", "Weightage": 0.0105},
    # {"Ticker": "TITAN.NS", "Weightage": 0.0137},
    # {"Ticker": "ULTRACEMCO.NS", "Weightage": 0.0102},
    # {"Ticker": "UPL.NS", "Weightage": 0.006},
    # {"Ticker": "WIPRO.NS", "Weightage": 0.0101},
]

In [73]:
def calculate_rsi(data, column="Adj Close", period=14):
    # Calculate price changes
    delta = data[column].diff()

    # Separate gains and losses
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # Calculate rolling averages of gains and losses
    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()

    # Compute RSI
    rs = avg_gain / avg_loss
    RSI = 100 - (100 / (1 + rs))

    return RSI

In [74]:
# Ensure start_date and end_date are timestamps
start_date = pd.Timestamp(start_date)
end_date = pd.Timestamp(end_date)

results = []

# Process each stock
for stock in stocks:
    ticker = stock["Ticker"]
    weightage = stock["Weightage"]

    try:
        # Fetch stock data
        data = yf.download(ticker, start=start_date, end=end_date)
        if data.empty:
            print(f"Data for {ticker} is not available. Skipping.")
            continue

        # Calculate RSI
        data["RSI"] = calculate_rsi(data, column="Adj Close", period=14)

        # Adjust weightage dynamically
        data["Adjusted Weightage"] = weightage
        data.loc[data["RSI"] > 90, "Adjusted Weightage"] -= 0.003
        data.loc[(data["RSI"] > 80) & (data["RSI"] <= 90), "Adjusted Weightage"] -= 0.002
        data.loc[(data["RSI"] > 70) & (data["RSI"] <= 80), "Adjusted Weightage"] -= 0.001
        data.loc[data["RSI"] < 20, "Adjusted Weightage"] += 0.003
        data.loc[(data["RSI"] >= 20) & (data["RSI"] < 30), "Adjusted Weightage"] += 0.002
        data.loc[(data["RSI"] >= 30) & (data["RSI"] <= 70), "Adjusted Weightage"] += 0.001

        # Calculate investment per stock
        investment_per_stock = investment_amount * weightage

        # Debug shapes
        print("Adjusted Weightage Shape:", data["Adjusted Weightage"].shape)
        print("Adj Close Shape:", data["Adj Close"].shape)

        # Ensure no NaN values
        data["Adjusted Weightage"].fillna(0, inplace=True)
        data["Adj Close"].fillna(1, inplace=True)  # Avoid division by zero

        # Calculate Shares
        data["Shares"] = investment_per_stock / data["Adj Close"]
        data["Adjusted Shares"] = (
            (investment_amount * data["Adjusted Weightage"]) / data["Adj Close"]
        )

        # Filter data for start_date and end_date
        filtered_data = data.loc[(data.index == start_date) | (data.index == end_date)]
        if filtered_data.empty:
            print(f"No data for {ticker} on {start_date.date()} or {end_date.date()}. Skipping.")
            continue

        # Append results for each filtered row
        for index, row in filtered_data.iterrows():
            try:
                results.append(
                    {
                        "Ticker": ticker,
                        "Date": index.date(),
                        "Weightage": weightage,
                        "Adjusted Weightage": row["Adjusted Weightage"],
                        "Open": row["Open"],
                        "Close": row["Adj Close"],
                        "RSI": row["RSI"],
                        "Shares": row["Shares"],
                        "Adjusted Shares": row["Adjusted Shares"],
                    }
                )
            except KeyError as ke:
                print(f"KeyError while processing row for {ticker}: {ke}")
            except Exception as e:
                print(f"Error processing row for {ticker}: {e}")

    except ZeroDivisionError as zde:
        print(f"ZeroDivisionError for {ticker}: {zde}")
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# Convert results to DataFrame
summary = pd.DataFrame(results)

# Debugging: Check the actual column names in summary
print("Column names in summary:", summary.columns)

# Reorder columns to match the output format
expected_columns = [
    "Ticker",
    "Date",
    "Weightage",
    "Adjusted Weightage",
    "Open",
    "Close",
    "RSI",
    "Shares",
    "Adjusted Shares",
]
missing_columns = [col for col in expected_columns if col not in summary.columns]

if missing_columns:
    print(f"Missing columns: {missing_columns}")
else:
    # Reorder columns if all are present
    summary = summary[expected_columns]

# Display the summary
print("\nInvestment Summary:")
print(summary)

# Optional: Save to CSV
# summary.to_csv("investment_summary.csv", index=False)
# print("\nSummary saved to 'investment_summary.csv'")


[*********************100%***********************]  1 of 1 completed


C:\Users\91994\AppData\Local\Temp\ipykernel_6636\3410298781.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Adjusted Weightage"].fillna(0, inplace=True)
C:\Users\91994\AppData\Local\Temp\ipykernel_6636\3410298781.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Adj Close"].fillna(1, inplace=True)  # Avoid division by zero


Adjusted Weightage Shape: (80,)
Adj Close Shape: (80, 1)
Error processing ADANIPORTS.NS: Cannot set a DataFrame with multiple columns to the single column Adjusted Shares
Column names in summary: RangeIndex(start=0, stop=0, step=1)
Missing columns: ['Ticker', 'Date', 'Weightage', 'Adjusted Weightage', 'Open', 'Close', 'RSI', 'Shares', 'Adjusted Shares']

Investment Summary:
Empty DataFrame
Columns: []
Index: []
